In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.13.1


In [3]:
x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
session = tf.Session()
session.run(x.initializer)
session.run(y.initializer)
result = session.run(f)
print(result)
session.close()

42


In [5]:
with tf.Session() as session:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [6]:
init = tf.global_variables_initializer()

with tf.Session() as session:
    init.run()
    result = f.eval()
    print(result)

42


In [7]:
session = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
session.close()

42


In [8]:
x1 = tf.Variable(1)

In [9]:
x1.graph

In [10]:
x2 = tf.Variable(2)

In [11]:
x2.graph

In [12]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [13]:
x2.graph

In [14]:
x2.graph is tf.get_default_graph()

False

In [15]:
tf.reset_default_graph()

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [17]:
with tf.Session() as session:
    print(y.eval()) #10
    print(z.eval()) #15

10
15


In [18]:
# evaluate w and x once
with tf.Session() as session:
    y_val, z_val = session.run([y, z])
    print(y_val) # 10
    print(z_val) # 15

10
15


In [19]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # theta = (XT * X)^(-1) * XT * y

with tf.Session() as session:
    theta_value = theta.eval()

In [20]:
theta_value

array([[-3.6959320e+01],
       [ 4.3698898e-01],
       [ 9.4245886e-03],
       [-1.0791138e-01],
       [ 6.4842808e-01],
       [-3.9986235e-06],
       [-3.7866351e-03],
       [-4.2142656e-01],
       [-4.3467718e-01]], dtype=float32)

In [21]:
from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

standard_scaler = StandardScaler()
scaled_housing_data_plus_bias = standard_scaler.fit_transform(housing_data_plus_bias)

In [22]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  6.3539624
Epoch 100 MSE =  5.106401
Epoch 200 MSE =  5.0134077
Epoch 300 MSE =  4.9620337
Epoch 400 MSE =  4.924595
Epoch 500 MSE =  4.89651
Epoch 600 MSE =  4.875299
Epoch 700 MSE =  4.8592033
Epoch 800 MSE =  4.8469296
Epoch 900 MSE =  4.8375216
Wall time: 893 ms


In [23]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  7.784878
Epoch 100 MSE =  4.9838405
Epoch 200 MSE =  4.882915
Epoch 300 MSE =  4.858183
Epoch 400 MSE =  4.843253
Epoch 500 MSE =  4.832592
Epoch 600 MSE =  4.824832
Epoch 700 MSE =  4.8191667
Epoch 800 MSE =  4.815021
Epoch 900 MSE =  4.811983
Wall time: 943 ms


In [24]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)

init = tf.global_variables_initializer()

Wall time: 39 ms


In [28]:
DATA_PATH = "D:/ML/Data/Home Credit Default Risk/"

In [30]:
import pandas as pd

In [33]:
df_train = pd.read_csv(DATA_PATH + "application_train.csv")

In [34]:
df_test = pd.read_csv(DATA_PATH + "application_test.csv")

In [35]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [36]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB


In [38]:
train_col = df_train.columns

In [39]:
test_col = df_test.columns

In [44]:
for col in train_col:
    if not col in test_col:
        print(col)

TARGET


In [47]:
df_train['TARGET']

0         1
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        1
27        0
28        0
29        0
         ..
307481    1
307482    0
307483    0
307484    0
307485    0
307486    0
307487    0
307488    0
307489    1
307490    0
307491    0
307492    0
307493    0
307494    0
307495    0
307496    0
307497    0
307498    0
307499    0
307500    0
307501    0
307502    0
307503    0
307504    0
307505    0
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64

In [48]:
y = df_train['TARGET']

In [51]:
X = df_train.drop(columns = 'TARGET')

In [52]:
print(X.columns)

Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=121)


In [53]:
X = tf.constant(X.drop, dtype = tf.float32, name = "X")
y = tf.constant(y.reshape(-1, 1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # theta = (XT * X)^(-1) * XT * y

with tf.Session() as session:
    theta_value = theta.eval()

TypeError: data type not understood